In [1]:
import torch
import torchvision.transforms as transforms              
from torchvision.datasets import CIFAR100      
from torch.utils.data import DataLoader
from src.temperature_scaling import ModelWithTemperature
from src.saps_hyp_opt import lambda_optimization_saps
from src.inception_cifar100 import inceptionv3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dict_path = "C:\\Users\\jiayang\\ipynb\\trainedModel\\Inception_CIFAR100.pth"
model = inceptionv3()
model.load_state_dict(torch.load(dict_path, map_location=device, weights_only=True))
model.to(device)

# preprocess the images from CIFAR10
data_transform = transforms.Compose([
    transforms.ToTensor(),         
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))  
])

# load data set from CIFAR100
dataset = CIFAR100(root="../../../data", train=False, download=True,transform=data_transform)

# Temperature Scaling
temp_scal_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ModelWithTemperature(model, temperature=0.5).to(device)
model.set_temperature(temp_scal_loader)
model.eval()

lambda_values = [0, 0.02, 0.03, 0.05, 0.08, 0.1, 0.13, 0.17, 0.2]

# lambda optimization
print("Looking for optimal lambda...")
optimal_lambda = lambda_optimization_saps(model, dataset, lambda_values, device)
if optimal_lambda is None:
    print("No optimal lambda is found")
else:
    print(f"Optimal lambda is {optimal_lambda}\n")

Files already downloaded and verified
Before temperature - NLL: 1.377, ECE: 0.118
Optimal temperature: 0.633
After temperature - NLL: 1.862, ECE: 0.200
Looking for optimal lambda...
Optimal lambda is 0.13

